# 라이브러리 불러오기

In [3]:
# ! pip install -U imbalanced-learn scikit-learn

In [3]:
import numpy as np
import pandas as pd
import re
import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')

from nltk.tokenize import TreebankWordTokenizer # 표준 토큰화
from nltk.corpus import stopwords # 불용어 제거
from nltk.stem import WordNetLemmatizer # 기본 형태로 변환
from imblearn.under_sampling import NeighbourhoodCleaningRule # 비대칭이라 사용

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\mit005\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mit005\AppData\Local\Temp\ipykernel_12004\2684078333.py", line 15, in <module>
    import tensorflow as tf
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1138, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 1078, in _find_spec
  File "<frozen importlib._bootstrap_external>", line 1504, in find_spec
  File "<frozen importlib._bootstrap_external>", line 1473, in _get_spec
TypeError: 'frame' object is not callable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mit005\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
      

# 데이터 불러오기

In [4]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
submission_df = pd.read_csv('./data/sample_submission.csv')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\mit005\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mit005\AppData\Local\Temp\ipykernel_12004\1359571154.py", line 1, in <module>
    train_df = pd.read_csv('./data/train.csv')
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mit005\anaconda3\Lib\site-packages\pandas\io\parsers\readers.py", line 912, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mit005\anaconda3\Lib\site-packages\pandas\io\parsers\readers.py", line 577, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mit005\anaconda3\Lib\site-packages\pandas\io\parsers\readers.py", line 1407, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
display(train_df.head())
display(test_df.head())

In [ ]:
train_df.groupby('first_party_winner').size()

# 데이터 전처리

In [ ]:
# 문자 처리
cat_cols = ['first_party', 'second_party', 'facts']

# \b : 단어 경계, W* : 길이가 0이상이고 단어가 아닌 문자, w{1} : 길이가 1인 단어
short_word = re.compile(r'\W*\b\w{1}\b') # 길이가 1인 단어 찾기
tokenizer = TreebankWordTokenizer() # 단어 단위로 토큰화
stopword = stopwords.words('english') # 불용어 리스트 가져오기
lemmatizer = WordNetLemmatizer() # 단어의 기본 형태 가져오기

In [ ]:
vec = CountVectorizer(ngram_range = (1, 2)) # 출현빈도
vec_facts = TfidfVectorizer(ngram_range = (1, 2)) # 단어토큰

## 단어 처리

In [ ]:
def prepro1(df, cols, short_word, tokenizer, stopword, lemmatizer):
    first_party_lst = []
    second_party_lst = []
    facts_lst = []
    
    for col in cols:
        df[col] = df[col].str.strip() # 공백 제거
        df[col] = df[col].str.lower() # 소문자로 변경
        df[col] = df[col].str.replace(',', '')
        df[col] = df[col].str.replace('.', '')
        
        if col == 'first_party':
            for content in df[col]:
                content = short_word.sub('', content) # 한 글자 단어 제거
                com = re.compile(r"[^\uAC00-\uD7A30-9a-zA-Z\s]") # 한글, 영어, 숫자 및 공백 문자를 제외한 모든 문자를 매칭
                content = com.sub('', content)
                tokens = tokenizer.tokenize(content) # 단어 토큰화
                token_lst = []
                for token in tokens:
                    if token not in stopword: #불용어 제거
                        token_lst.append(lemmatizer.lemmatize(token, 'n')) # 단어의 기본 형태 가져오기
                first_party_lst.append(token_lst)
            # 단어들 결합
            for i in range(len(first_party_lst)):
                first_party_lst[i] = ' '.join(first_party_lst[i])
                
        elif col == 'second_party':
            for content in df[col]:
                content = short_word.sub('', content) # 한 글자 단어 제거
                com = re.compile(r"[^\uAC00-\uD7A30-9a-zA-Z\s]") # 한글, 영어, 숫자 및 공백 문자를 제외한 모든 문자를 매칭
                content = com.sub('', content)
                tokens = tokenizer.tokenize(content) # 단어 토큰화
                token_lst = []
                for token in tokens:
                    if token not in stopword: #불용어 제거
                        token_lst.append(lemmatizer.lemmatize(token, 'n')) # 단어의 기본 형태 가져오기
                second_party_lst.append(token_lst)
            # 단어들 결합
            for i in range(len(second_party_lst)):
                second_party_lst[i] = ' '.join(second_party_lst[i])
                
        elif col == 'facts':
            for content in df[col]:
                content = short_word.sub('', content) # 한 글자 단어 제거
                com = re.compile(r"[^\uAC00-\uD7A30-9a-zA-Z\s]") # 한글, 영어, 숫자 및 공백 문자를 제외한 모든 문자를 매칭
                content = com.sub('', content)
                tokens = tokenizer.tokenize(content) # 단어 토큰화
                token_lst = []
                for token in tokens:
                    if token not in stopword: #불용어 제거
                        token_lst.append(lemmatizer.lemmatize(token, 'n')) # 단어의 기본 형태 가져오기
                facts_lst.append(token_lst)
            # 단어들 결합
            for i in range(len(facts_lst)):
                facts_lst[i] = ' '.join(facts_lst[i])
                
    return first_party_lst, second_party_lst, facts_lst

## 벡터화

In [ ]:
def prepro2(first, second, facts, vec, vec_facts, is_train):
    if is_train:
        vec.fit(first + second) # conut
        vec_facts.fit(facts) # Tf
    
    X_first = vec.transform(first).toarray()
    X_second = vec.transform(first).toarray()
    X_facts = vec_facts.transform(facts).toarray()
    
    return np.concatenate([X_first, X_second, X_facts], axis = 1)

전처리 실행

In [ ]:
train_first, train_second, train_facts = prepro1(train_df, cat_cols, short_word, tokenizer, stopword, lemmatizer)
test_first, test_second, test_facts = prepro1(test_df, cat_cols, short_word, tokenizer, stopword, lemmatizer)

In [ ]:
X_train = prepro2(train_first, train_second, train_facts, vec, vec_facts, True)
y_train = train_df['first_party_winner']
X_test = prepro2(test_first, test_second, test_facts, vec, vec_facts, False)

In [ ]:
print('Train shape : {}, {}'.format(X_train.shape, y_train.shape))
print('Test shape : {}'.format(X_test.shape))

## 불균형 데이터 처리

NeighbourhoodCleaningRule

- 불균형한 데이터셋에서 소수 클래스의 근접한 이웃을 고려하여 데이터를 정제하는 언더 샘플링 기법 중 하나

1. 소수 클래스의 샘플을 중심으로 주변 이웃들을 찾는다

2. 이웃들 중에서 다수 클래스에 속한 샘플들을 제거한다

3. 이렇게 제거된 샘플들을 제외하고 남은 샘플들을 반환한다

In [12]:
X_ncr, y_ncr = NeighbourhoodCleaningRule(kind_sel = "all",  n_neighbors = 5).fit_resample(X_train, y_train)
print('train shape : {}, {}'.format(X_ncr.shape, y_ncr.shape))

C:\Users\user\anaconda3\Lib\site-packages\imblearn\under_sampling\_prototype_selection\_neighbourhood_cleaning_rule.py:201: FutureWarning: `kind_sel` is deprecated in 0.12 and will be removed in 0.14. It already has not effect and corresponds to the `'all'` option.
  warnings.warn(


train shape : (1775, 204261), (1775,)


In [13]:
y_ncr.value_counts()

first_party_winner
1    946
0    829
Name: count, dtype: int64

# 모델링

## 데이터 분리

train데이터 셋으로 train, val 데이터 셋으로 분리

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X_ncr, y_ncr, random_state = 42, test_size = 0.2, 
                                                 stratify = y_ncr)

print('Train shape : {}, {}'.format(X_train.shape, y_train.shape))
print(y_train.value_counts())

print()

print('Validation shape : {}, {}'.format(X_val.shape, y_val.shape))
print(y_val.value_counts())

Train shape : (1420, 204261), (1420,)
first_party_winner
1    757
0    663
Name: count, dtype: int64

Validation shape : (355, 204261), (355,)
first_party_winner
1    189
0    166
Name: count, dtype: int64


In [15]:
# LSTM 모델 생성
model = Sequential()

vocab_size = 10000
embedding_dim = 100

# 임베딩 레이어 추가
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))

# LSTM 레이어 추가 (return_sequences=False로 설정)
model.add(LSTM(units=64))

# Dense 레이어 추가
model.add(Dense(units=64, activation='relu'))

# 출력 레이어 추가
model.add(Dense(units=1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val))


Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node sequential_1/lstm_1/TensorArrayUnstack/TensorListFromTensor defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\user\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\user\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "C:\Users\user\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\user\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\Users\user\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\Users\user\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "C:\Users\user\anaconda3\Lib\asyncio\events.py", line 80, in _run

  File "C:\Users\user\anaconda3\Lib\site-packages\ipykernel\kernelbase.py", line

# 제출

In [ ]:
# y_pred  = best_logistic.predict(X_test)
# submission_df['first_party_winner'] = y_pred
# submission_df.to_csv('neighbourhoodcleaningrule_logi.csv', index = False)